In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

from hlpr_func import make_cyclic

In [ ]:
# ceres insolation and clear-sky fluxes
ceres = make_cyclic(xr.open_mfdataset("CERES-data/CERES_Cru23_ds.nc"))
ceres = ceres[["S"]]

# reanalyses
era = xr.open_dataset("ERA5-CAMS/era5-interpolated.nc")
era = era[["landmask","si10"]]
era = era.rename_vars({"si10":"era_ws"})

merra_ws = xr.open_dataset("MERRA-2/merra2-ws-interpolated.nc")
merra_ws = merra_ws[["SPEED"]].rename_vars({"SPEED":"merra_ws"})

ds = xr.merge([ceres, era, merra_ws])
ceres.close()
era.close()
merra_ws.close()

ds

In [ ]:
# add extra coord
da = ds.assign_coords({"month_since_start": ("time", np.arange(len(ds.time.values)))})

# select only DJF
djf = da.where(ds.time.dt.month.isin([12,1,2]))

# remove austrailian bush fires (Dec 2019 - Mar 2020)
djf = djf.where(~(djf.time.dt.year.isin(2019) & djf.time.dt.month.isin(12)))
djf = djf.where(~(djf.time.dt.year.isin(2020) & djf.time.dt.month.isin([1,2])))

# add weights
weights = djf.time.dt.days_in_month
weights = weights.where(weights.time.dt.month!=2, 28.65)
djf["days_in_month"] = weights

# calculate climatology and remove seasons
climatology = djf.groupby("time.month").mean("time")
deseasonalized = djf.groupby("time.month") - climatology + climatology.weighted(climatology.days_in_month).mean("month")

# select Southern Ocean
so = deseasonalized.sel(lat=slice(-75,-45)).where(deseasonalized.landmask==0)
so["days_in_month"] = weights


# swap time to monotonic index
da = so.swap_dims({"time":"month_since_start"})

# calculate linear trend
polyfit_result = da.polyfit("month_since_start", 1, skipna=True,  w=da.days_in_month)
trend = polyfit_result.sel(degree=1) * 120 # per decade
for v in trend.keys():
    trend = trend.rename_vars({v:v.split("_polyfit")[0]})

trend

In [ ]:
fig,axes = plt.subplots(1,2,figsize=(10,5),constrained_layout=True)

ax = axes[0]
ax.plot(so.era_ws.values.flatten(), so.merra_ws.values.flatten(), "k.", alpha=0.5)
rmse = np.sqrt(np.nanmean((so.merra_ws.values - so.era_ws.values)**2))
ax.text(2, 16, f"RMSE = {rmse:.3f}")
ax.plot([1.5,17],[1.5,17], "k-")
ax.set_xlim([1.5,17])
ax.set_ylim([1.5,17])
ax.set_xlabel("ERA5")
ax.set_ylabel("MERRA-2")
ax.set_title("Wind speed [m s$^{-1}$]")
ax.set_title("a)", loc="left")

ax = axes[1]
ax.plot(trend.era_ws.values.flatten(), trend.merra_ws.values.flatten(), "k.", alpha=0.5)
rmse = np.sqrt(np.nanmean((trend.merra_ws.values - trend.era_ws.values)**2))
ax.text(-0.8, 0.8, f"RMSE = {rmse:.3f}")
ax.plot([-0.85,0.9],[-0.85,0.9], "k-")
ax.set_xlim([-0.85,0.9])
ax.set_ylim([-0.85,0.9])
ax.set_xlabel("ERA5")
ax.set_ylabel("MERRA-2")
ax.set_title("Wind speed trend [m s$^{-1}$ decade$^{-1}$]")
ax.set_title("b)", loc="left")

plt.savefig("paper-figures/era-merra-ws.png", dpi=200, facecolor="w", bbox_inches="tight")
plt.show()